In [1]:
import os
import os.path
import re
import sys
sys.path.append("src")

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 1000)
import datetime
import matplotlib.pylab as plt
%matplotlib inline
import seaborn as sns
sns.set_style('whitegrid')
import time
import copy

import rqalpha.api as rqa
import tensorflow as tf
from rqalpha import run_func
from Talib_calc import *
from Classifier_PonderDNC_BasicLSTM_L3 import *


def init(context):
    context.stock_list = ['000001.XSHE', '000002.XSHE', '000008.XSHE', '000009.XSHE', '000060.XSHE']
    context.BarSpan = 200
    context.TransactionRate = '1d'
    context.DataFields = ['datetime', 'open', 'close','high', 'low', 'volume']
    context.DefineQuantity = 5 
    context.func_get_factors = get_factors
    context.model_dic = {}
    context.sess_dic = {}
    context.graph_dic = {}
    
    for stock in context.stock_list:
        print(stock)
        
        g = tf.Graph()
        sess = tf.Session(graph = g)
        
        with sess.as_default():
            with g.as_default():
                
                tf.global_variables_initializer().run()
                
                inputs_0 = np.zeros((619,1,58))
                targets_0 = np.zeros((608,1,3))
                gather_list_0 = np.zeros(1,dtype=np.int)
                
                op = Classifier_PonderDNC_BasicLSTM_L3(
                    inputs= inputs_0, 
                    targets= targets_0, 
                    gather_list= gather_list_0, 
                    hidden_size= 50, 
                    memory_size= 50, 
                    pondering_coefficient= 1e-1, 
                    learning_rate= 1e-4)

                model_path =  'test/'+ stock + '.csv/ResidualPonderDNC.ckpt'
                # 模型预测
                op.load_model(model_path)
                
                op.close()
        
                context.model_dic[stock] = op
                context.sess_dic[stock] = sess
                context.graph_dic[stock] = g
                
        pass

         when using: `python -m better_exceptions'. Otherwise, only code
         loaded from files will be inspected!


In [2]:
def handle_bar(context, bar_dict): 
    
    for stock in context.stock_list:
        
        g = context.graph_dic[stock]
        sess = context.sess_dic[stock]
        print(g, sess)

        # 加载第一个模型
        with sess.as_default(): 
            with g.as_default():
                
                op = context.model_dic[stock] 
                
                model_path =  'test/'+ stock + '.csv/ResidualPonderDNC.ckpt'
                # 模型预测
                op.load_model(model_path, sess)
        
                # 获取合约报价
                Quotes = rqa.history_bars(
                    order_book_id= stock, 
                    bar_count= context.BarSpan, 
                    frequency= context.TransactionRate,
                    fields= context.DataFields)
                Quotes = pd.DataFrame(Quotes)

                # 计算技术分析指标
                tmp_factors = context.func_get_factors(
                    index= pd.to_datetime(Quotes['datetime']), 
                    Open= Quotes['open'].values, 
                    Close= Quotes['close'].values, 
                    High= Quotes['high'].values, 
                    Low= Quotes['low'].values, 
                    Volume=Quotes['volume'].values,
                    drop=True)   
                inputs = np.expand_dims(np.array(tmp_factors), axis=1)  
                probability, classification = op.pred(inputs)

                flag = classification[-1][0]
                #print(classification[-1], probability[-1])
        
                #op.close()
        rqa.logger.info(str(flag))
    
        # 绘制估计概率
        #rqa.plot("估空概率", probability[-1][0][0])
        #rqa.plot("振荡概率", probability[-1][0][1])
        #rqa.plot("估多概率", probability[-1][0][2])
     
        # 获取仓位
        cur_position = context.portfolio.accounts['STOCK'].positions

        # 卖出
        if flag == 0:
            #rqa.logger.info ('沽空')
            rqa.order_target_percent(stock, 0)            
            
        # 买入
        if flag == 2:
            #rqa.logger.info ('沽多')
            rqa.order_target_percent(stock, 0.5)      
            
        if flag == 1:
            '''
            rqa.logger.info ('振荡区间')
            if tmp_sell_quantity > 0:
                rqa.buy_close(contract, tmp_sell_quantity)
                rqa.logger.info ('平空单')
            if tmp_buy_quantity > 0:
                rqa.sell_close(contract, tmp_buy_quantity)
                rqa.logger.info ('平多单')
            else:
                rqa.logger.info ('空仓规避')
            '''

In [3]:
start_date = '2017-01-01'
end_date = '2017-8-27'
accounts = {'stock':1e5}

config = {
    'base':{'start_date':start_date, 'end_date':end_date, 'accounts':accounts},
    'extra':{'log_level':'info'},
    'mod':{'sys_analyser':{'enabled':True, 'plot':True}}
}

results = run_func(init=init, handle_bar=handle_bar, config=config)

000001.XSHE


/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:95: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


AttributeError: 'OutStream' object has no attribute 'buffer'